In [19]:
import numpy as np
import networkx as nx
import json
import copy
import sys
import os
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed
from pythreejs import *
import math
sys.path.insert(0, os.path.abspath('..'))

In [10]:
class CubeBase:
    tables = None
    piece_initial_positions = np.array([
        [[1 , 2 , 3 ],
         [4 , 5 , 6 ],
         [7 , 8 , 9 ]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]],

        [[19, 20, 21],
         [22, 23, 24],
         [25, 26, 27]],
    ])
    piece_initial_orientations = np.array([
        [['xyZ', 'g', 'XyZ'],
         ['g'  , 'y', 'g'  ],
         ['xyz', 'g', 'Xyz']],

        [['g'  , 'Z', 'g'  ],
         ['x'  , 'C', 'X'  ],
         ['g'  , 'z', 'g'  ]],

        [['xYZ', 'g', 'XYZ'],
         ['g'  , 'Y', 'g'  ],
         ['xYz', 'g', 'XYz']],
    ])
    @classmethod
    def initialize(cls):
        if cls.tables is None:
            cls.edge_positions, cls.corner_positions = cls.categorize_positions_over_piece_types()
            cls.edge_ids, cls.corner_ids = cls.categorize_ids_over_piece_types()

            cls.edge_positions.sort()
            cls.corner_positions.sort()
            cls.edge_ids.sort()
            cls.corner_ids.sort()

            cls.tables = cls._load_tables_from_json([
                    '../Precomputed_Tables/corner_position_distance_table.json',
                    '../Precomputed_Tables/edge_position_distance_table.json',
                    '../Precomputed_Tables/position_movement_table.json'
            ])
            cls.edge_distances = cls.tables["edge_distances"]
            cls.corner_distances = cls.tables["corner_distances"]
            cls.movements = cls.tables["movements"]

    @classmethod
    def categorize_ids_over_piece_types(cls):
        """Identifies edge and corner pieces based on orientation markers."""
        edge_ids = []
        corner_ids = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = cls.piece_initial_positions[i, j, k]
                    if piece_id in [5, 11, 13, 14, 15, 17, 23]:
                        continue
                    orientation = cls.piece_initial_orientations[i, j, k]
                    if orientation == 'g':
                        edge_ids.append(piece_id)
                    else:
                        corner_ids.append(piece_id)
        return edge_ids, corner_ids
    
    @classmethod
    def categorize_positions_over_piece_types(cls):
        """ Iterate through all positions in the cube and sort their positions into edges and corners """
        edge_positions = []
        corner_positions = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = cls.piece_initial_positions[i, j, k]
                    if piece_id in [5, 11, 13, 14, 15, 17, 23]:
                        continue
                    orientation = cls.piece_initial_orientations[i, j, k]
                    if orientation == 'g':
                        edge_positions.append((i, j, k))
                    else:
                        corner_positions.append((i, j, k))
        return edge_positions, corner_positions

    @staticmethod
    def _load_tables_from_json(filenames: list):
        """
        Loads precomputed tables from JSON files and returns them in a dictionary.

        Args:
            filenames: List of JSON filenames containing the precomputed tables

        Returns:
            dict: A dictionary containing loaded tables, with keys: "edge_distances", "corner_distances", "movements".
            Values are the loaded tables, or None if loading failed for a table type.
        """
        tables = {
            "edge_distances": None,
            "corner_distances": None,
            "movements": None
        }

        for filename in filenames:
            try:
                with open(filename, 'r') as f:
                    serializable_table = json.load(f)
                    
                    # Determine which table type this file contains
                    if 'edge' in filename.lower() and 'distance' in filename.lower():
                        tables["edge_distances"] = {}
                        for pair_str, distance in serializable_table.items():
                            pos_tuple = tuple(eval(pair_str))  # Consistent parsing using eval
                            tables["edge_distances"][pos_tuple] = distance
                            
                    elif 'corner' in filename.lower() and 'distance' in filename.lower():
                        tables["corner_distances"] = {}
                        for pair_str, distance in serializable_table.items():
                            pos_tuple = tuple(eval(pair_str))  # Consistent parsing
                            tables["corner_distances"][pos_tuple] = distance
                            
                    elif 'position' in filename.lower() and 'movement' in filename.lower():
                        tables["movements"] = {}
                        for move, position_movements in serializable_table.items():
                            movements = {}
                            for from_pos_str, to_pos_str in position_movements.items():
                                from_pos = tuple(eval(from_pos_str))
                                to_pos = tuple(eval(to_pos_str))
                                movements[from_pos] = to_pos
                            tables["movements"][move] = movements
                            
            except FileNotFoundError:
                print(f"FileNotFoundError: '{filename}' not found.")
            except json.JSONDecodeError:
                print(f"JSONDecodeError: Could not decode JSON from '{filename}'. File may be corrupted.")
            except Exception as e:
                print(f"Error loading '{filename}': {e}")
        
        # Log which tables were successfully loaded
        loaded_tables = [key for key, value in tables.items() if value is not None]
        if loaded_tables:
            print(f"Successfully loaded tables: {', '.join(loaded_tables)}")
        else:
            print("Warning: No tables were successfully loaded")
            
        return tables
    
class CubeTracker(CubeBase):
    def __init__(self):
        CubeBase.initialize()
        self.piece_current_positions = copy.deepcopy(CubeBase.piece_initial_positions)
        self.piece_current_orientations = copy.deepcopy(CubeBase.piece_initial_orientations)
    
        self.move_map = {
                'L': self._L, 'l': self._l, 'R': self._R, 'r': self._r,
                'F': self._F, 'f': self._f, 'B': self._B, 'b': self._b,
                'U': self._U, 'u': self._u, 'D': self._D, 'd': self._d,
                'N': self._N
        }
        # The uppercase letters are the clockwise moves, and the lowercase letters are the counter-clockwise moves

        self.corner_move_vs_facelet_swap_map = {
            'L': ((1,2),'x'), 'l': ((1,2),'x'), 'R': ((1,2),'x'), 'r': ((1,2),'x'),
            'F': ((0,2),'y'), 'f': ((0,2),'y'), 'B': ((0,2),'y'), 'b': ((0,2),'y'),
            'U': ((0,1),'z'), 'u': ((0,1),'z'), 'D': ((0,1),'z'), 'd': ((0,1),'z'),
            'N': ((0,0),'x')
        }

    def _rotate_face(self, perspective, face_idx, direction):
        """ Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction """
        def change_perspective(cube, perspective, direction):
            if perspective == 0: return cube
            else: return np.rot90(cube, k=direction, axes=(0, perspective))
        # Convert to the desired perspective, rotate the face, then convert back
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        self.piece_current_positions[face_idx] = np.rot90(self.piece_current_positions[face_idx], k=direction, axes=(0, 1))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)

        self.piece_current_orientations = change_perspective(self.piece_current_orientations, perspective, -1)
        self.piece_current_orientations[face_idx] = np.rot90(self.piece_current_orientations[face_idx], k=direction, axes=(0, 1))
        self.piece_current_orientations = change_perspective(self.piece_current_orientations, perspective, 1)

    def _L(self): self._rotate_face(2, 0, -1)
    def _l(self): self._rotate_face(2, 0,  1)
    def _R(self): self._rotate_face(2, 2,  1)
    def _r(self): self._rotate_face(2, 2, -1)
    def _F(self): self._rotate_face(0, 0, -1)
    def _f(self): self._rotate_face(0, 0,  1)
    def _B(self): self._rotate_face(0, 2,  1)
    def _b(self): self._rotate_face(0, 2, -1)
    def _U(self): self._rotate_face(1, 0, -1)
    def _u(self): self._rotate_face(1, 0,  1)
    def _D(self): self._rotate_face(1, 2,  1)
    def _d(self): self._rotate_face(1, 2, -1)
    def _N(self): pass

    def map_moves_to_inverse_moves(self):
        all_moves = self.move_map.keys()
        moves = []
        inverse_moves = []
        for move in all_moves:
            if move.isupper():
                moves.append(move)
            else:
                inverse_moves.append(move)
        moves.remove('N')
        moves.sort()
        inverse_moves.sort()
        return list(zip(moves, inverse_moves))
        
    def _get_position_of_piece(self, piece_id):
        """Returns the 3D position vector (tuple) of a piece given the piece_id"""
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return (i, j, k)

    def _get_piece_at_position(self, position):
        """Returns the piece ID at a given position (i, j, k)."""
        i, j, k = position
        return self.piece_current_positions[i, j, k]
    
    def _get_orientation_of_piece(self, piece_id):
        """Returns the orientation of a piece given its ID."""
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return self.piece_current_orientations[i, j, k]
    
    def _update_edge_orientations(self, move):
        """Updates the orientations of edges based on the move made """
        for edge in self.edge_positions:
            if move in self.movements.keys():
                if edge in self.movements[move].keys():
                    piece_id = self.piece_current_positions[edge]
                    piece_initial_position = tuple(np.argwhere(self.piece_initial_positions == piece_id)[0])
                    if self.edge_distances[(piece_initial_position, edge)] == self.edge_distances[(piece_initial_position, self.movements[move][edge])]:
                        current_orientation = self.piece_current_orientations[edge]
                        self.piece_current_orientations[edge] = 'g' if current_orientation=='b' else 'b'
    
    def _update_corner_orientations(self, move):
        """Updates the orientations of corners based on the move made """

        def remove(lst, item):
            return [x for x in lst if x != item]
        
        for corner in self.corner_positions:
            if move in self.movements.keys():
                if corner in self.movements[move].keys():
                    current_orientation = list(self.piece_current_orientations[corner])
                    final_corner_orientation = list(self.piece_initial_orientations[CubeBase.movements[move][corner]])
                    reference_orientation = list(self.piece_initial_orientations[corner].lower())
                    reference_constant_facelet_id = self.corner_move_vs_facelet_swap_map[move][1]
                    reference_constant_facelet = reference_orientation.index(reference_constant_facelet_id)
                    corner_constant_facelet = ''.join(current_orientation).lower().index(reference_constant_facelet_id)
                    final_corner_constant_facelet_id = final_corner_orientation[reference_constant_facelet]
                    corner_facelets_to_swap = remove(list(range(0, len(reference_orientation))), corner_constant_facelet)
                    corner_facelet_ids_to_swap = remove(current_orientation, current_orientation[corner_constant_facelet])
                    
                    zipped = list(zip(corner_facelet_ids_to_swap, corner_facelets_to_swap))
                    new_orientation = list(range(0,3))
                    count = 1
                    for id, facelet in zipped:
                        if count == 1:
                            next = final_corner_orientation.pop(reference_orientation.index(id.lower()))
                            final_corner_orientation.remove(final_corner_constant_facelet_id)
                            new_orientation[facelet] = final_corner_orientation[0]
                        else:
                            new_orientation[facelet] = next
                        if count == 2:
                            break
                        count += 1
    
                    new_orientation[corner_constant_facelet] = final_corner_constant_facelet_id
                    new_orientation = ''.join(new_orientation)
                    self.piece_current_orientations[corner] = new_orientation
                
    def apply_moves(self, move_sequence):
        """Applies the moves to the cube state (piece_current_positions and piece_current_orientations)
        Args:
            move_sequence(list/str): ordered set of moves as a list or a string
        """
        if not isinstance(move_sequence, (list, str)):
            raise ValueError("argument to apply_moves must be a list or a string of valid moves")

        if isinstance(move_sequence, str):
            move_sequence = list(move_sequence) # Convert string to list for consistent iteration

        for index, move in enumerate(move_sequence):
            if move in self.move_map:
                self._update_corner_orientations(move)
                self._update_edge_orientations(move)
                self.move_map[move]()
            else:
                raise ValueError(f"Invalid move: '{move}' at index {index}") # More readable error message

In [ ]:
class InteractiveRubiksCube:
    def __init__(self):
        # Scene setup
        self.scene = Scene(background="#404040")  
        self.camera = PerspectiveCamera(
            position=[3.5, 3.5, 5], 
            up=[0, 1, 0],
            aspect=1, 
            fov=40
        )
        self.renderer = Renderer(
            scene=self.scene, 
            camera=self.camera, 
            antialias=True, 
            width=600, 
            height=600
        )

        # Cubelet properties
        self.cubelet_size = 0.95
        self.cubelet_spacing = 0.2
        self.face_colors = {
            'W': '#FFFFFF',
            'Y': '#FFFF00',
            'G': '#00FF00',
            'B': '#0000FF',
            'R': '#FF0000',
            'O': '#FF8800',
            'k': '#222222'
        }

        # Mapping of standard color positions in solved state
        self.standard_colors = {
            'R': [1, 0, 0],   # Right face (+X)
            'L': [-1, 0, 0],  # Left face (-X)
            'U': [0, 1, 0],   # Up face (+Y)
            'D': [0, -1, 0],  # Down face (-Y)
            'F': [0, 0, 1],   # Front face (+Z)
            'B': [0, 0, -1]   # Back face (-Z)
        }

        # Standard color mapping
        self.color_map = {
            'R': 'R',  # Right face is red
            'L': 'O',  # Left face is orange 
            'U': 'W',  # Up face is white
            'D': 'Y',  # Down face is yellow
            'F': 'G',  # Front face is green
            'B': 'B'   # Back face is blue
        }

        self.pieces = {}
        self._create_cube()

        # Lighting
        self.scene.add(AmbientLight(color='#FFFFFF', intensity=0.6))
        for position in [[1, 1, 1], [-1, -1, 1], [1, -1, -1]]:
            directional_light = DirectionalLight(color='#FFFFFF', intensity=0.4)
            directional_light.position = position
            self.scene.add(directional_light)

        # Controls
        self.control = OrbitControls(controlling=self.camera)
        self.control.target = [0, 0, 0]
        self.renderer.controls = [self.control]

    def _create_cube(self):
        piece_positions_initial = np.array([
            [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
            [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
            [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
        ])

        # Create a mapping of each piece ID to its initial facelets and colors
        initial_facelets = {}
        
        # Corner pieces have 3 facelets
        corners = [
            (1, ['U', 'L', 'F']),   # ULF
            (3, ['U', 'R', 'F']),   # URF
            (7, ['D', 'L', 'F']),   # DLF
            (9, ['D', 'R', 'F']),   # DRF
            (19, ['U', 'L', 'B']),  # ULB
            (21, ['U', 'R', 'B']),  # URB
            (25, ['D', 'L', 'B']),  # DLB
            (27, ['D', 'R', 'B'])   # DRB
        ]
        
        # Edge pieces have 2 facelets
        edges = [
            (2, ['U', 'F']),   # UF
            (4, ['L', 'F']),   # LF
            (6, ['R', 'F']),   # RF
            (8, ['D', 'F']),   # DF
            (10, ['U', 'L']),  # UL
            (12, ['U', 'R']),  # UR
            (16, ['D', 'L']),  # DL
            (18, ['D', 'R']),  # DR
            (20, ['U', 'B']),  # UB
            (22, ['L', 'B']),  # LB
            (24, ['R', 'B']),  # RB
            (26, ['D', 'B'])   # DB
        ]
        
        # Center pieces have 1 facelet
        centers = [
            (5, ['F']),    # F center
            (11, ['U']),   # U center
            (13, ['L']),   # L center
            (15, ['R']),   # R center
            (17, ['D']),   # D center
            (23, ['B'])    # B center
        ]
        
        # Combine all piece definitions
        for piece_id, faces in corners + edges + centers:
            initial_facelets[piece_id] = faces
        
        # Create all cube pieces
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = piece_positions_initial[i, j, k]
                    if piece_id == 14:  # Skip center piece
                        continue

                    # Create geometry and material
                    geometry = BoxGeometry(width=self.cubelet_size, 
                                         height=self.cubelet_size, 
                                         depth=self.cubelet_size)
                    
                    # Create a material for each face (right, left, up, down, front, back)
                    materials = []
                    for _ in range(6):
                        materials.append(MeshStandardMaterial(
                            color=self.face_colors['k'],  # Default black
                            roughness=0.7,
                            metalness=0.1
                        ))
                    
                    # Position of the piece
                    position = [
                        (k - 1) * (self.cubelet_size + self.cubelet_spacing),
                        (1 - j) * (self.cubelet_size + self.cubelet_spacing),
                        (1 - i) * (self.cubelet_size + self.cubelet_spacing)
                    ]
                    
                    # Create the mesh with geometry, materials, and position
                    mesh = Mesh(geometry=geometry, material=materials, position=position)
                    
                    # Store the piece's facelets for reference
                    if piece_id in initial_facelets:
                        mesh.userData = {
                            'facelets': initial_facelets[piece_id],
                            'piece_type': len(initial_facelets[piece_id])  # 3=corner, 2=edge, 1=center
                        }
                    
                    # Apply initial colors based on position
                    self._update_piece_colors(mesh, None)  # No rotation initially
                    
                    # Add to scene and store in pieces dictionary
                    self.scene.add(mesh)
                    self.pieces[piece_id] = mesh
    
    def _update_piece_colors(self, piece_mesh, quaternion):
        """Update colors of a piece based on its quaternion rotation"""
        # Get the piece's facelets
        if 'facelets' not in piece_mesh.userData:
            return
            
        facelets = piece_mesh.userData['facelets']
        
        # Face indices in the material array
        face_indices = {
            'R': 0,  # Right face (+X)
            'L': 1,  # Left face (-X)
            'U': 2,  # Up face (+Y)
            'D': 3,  # Down face (-Y)
            'F': 4,  # Front face (+Z)
            'B': 5   # Back face (-Z)
        }
        
        # Reset all faces to black
        for i in range(6):
            piece_mesh.material[i].color = self.face_colors['k']
        
        # Set colors for each facelet
        for facelet in facelets:
            # Get standard direction of this facelet
            direction = self.standard_colors[facelet]
            
            # If quaternion provided, rotate the direction
            if quaternion:
                # Apply rotation to direction vector (would use quaternion math here)
                # This is a simplified example - proper quaternion rotation needed
                pass
            
            # Determine which face this facelet is now on
            # For now, we'll simply use the original face mapping (this would change with rotation)
            face_idx = face_indices[facelet]
            color_code = self.color_map[facelet]
            
            # Set the color
            piece_mesh.material[face_idx].color = self.face_colors[color_code]

    def update_cube_from_tracker(self, cube_tracker):
        """Update the 3D cube based on the current state of a CubeTracker object using quaternions"""
        
        # Map for orientation encoding to face directions
        orientation_to_axis = {
            'x': [1, 0, 0],   # X-axis
            'y': [0, 1, 0],   # Y-axis 
            'z': [0, 0, 1]    # Z-axis
        }
        
        # Update piece positions and orientations
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    pos = (i, j, k)
                    piece_id = cube_tracker.piece_current_positions[i, j, k]
                    if piece_id == 14:  # Skip center piece
                        continue
                    
                    # Update position
                    self.pieces[piece_id].position = [
                        (k - 1) * (self.cubelet_size + self.cubelet_spacing),
                        (1 - j) * (self.cubelet_size + self.cubelet_spacing),
                        (1 - i) * (self.cubelet_size + self.cubelet_spacing)
                    ]
                    
                    # Get current orientation
                    orientation = cube_tracker.piece_current_orientations[i, j, k]
                    
                    # Convert orientation to quaternion for corners
                    if len(orientation) == 3:  # Corner piece
                        # Parse the orientation string
                        quaternion = self._orientation_string_to_quaternion(orientation)
                        if quaternion:
                            self.pieces[piece_id].quaternion = quaternion
                        pass
                    # For edges - simplified handling
                    elif orientation in ['g', 'b']:
                        # We would need more complex logic to handle edge orientations
                        pass
                    
                    # For centers - no orientation changes needed
                    else:
                        pass
    def _orientation_string_to_quaternion(self, orientation_string):
        pass
        
    def display(self):
        return self.renderer


# Test function to demonstrate the visualization with quaternions
def test_quaternion_cube():
    # Initialize the tracker and visualization
    cube_tracker = CubeTracker()
    interactive_cube = InteractiveRubiksCube()
    
    # Define a sequence of moves to apply
    moves = "L F R U B D l f r u b d"
    move_list = moves.split()
    
    # Create interactive widget
    move_dropdown = widgets.Dropdown(
        options=list(cube_tracker.move_map.keys()),
        value='L',
        description='Move:',
    )
    
    def apply_move(move):
        if move in cube_tracker.move_map:
            cube_tracker.apply_moves(move)
            interactive_cube.update_cube_from_tracker(cube_tracker)
        else:
            print(f"Invalid move: {move}")
    
    apply_button = widgets.Button(description='Apply Move')
    apply_button.on_click(lambda b: apply_move(move_dropdown.value))
    
    reset_button = widgets.Button(description='Reset Cube')
    reset_button.on_click(lambda b: (cube_tracker.__init__(), 
                                    interactive_cube.update_cube_from_tracker(cube_tracker),
                                    print("Cube reset")))
    
    # Add buttons for test sequences
    def apply_test_sequence(sequence):
        for move in sequence.split():
            cube_tracker.apply_moves(move)
            print(f"Applied move: {move}")
        interactive_cube.update_cube_from_tracker(cube_tracker)
        print(f"Corner (0,0,0) final orientation: {cube_tracker.piece_current_orientations[0,0,0]}")
    
    test_sequences = {
        "L' (l)": "l",
        "L'R'": "lr",
        "F'B'": "fb", 
        "U'D'": "ud"
    }
    
    sequence_buttons = []
    for name, sequence in test_sequences.items():
        btn = widgets.Button(description=name)
        btn.on_click(lambda b, seq=sequence: apply_test_sequence(seq))
        sequence_buttons.append(btn)
    
    ui = widgets.VBox([
        interactive_cube.display(),
        widgets.HBox([move_dropdown, apply_button, reset_button]),
        widgets.HBox(sequence_buttons)
    ])
    
    return ui

# Run in a Jupyter notebook
if __name__ == '__main__':
    display(test_quaternion_cube())